# Assignment 5

### Teaching Assistants:
- Wouter Bulten: wouter.bulten@radboudumc.nl
- David Tellez Martin: David.TellezMartin@radboudumc.nl

Please submit your notebook via grand-challenge.org (https://ismi-cifar.grand-challenge.org/introduction/).
Submit a notebook **WITH ALL CELLS EXECUTED!!!**

* Groups: You should work in pairs or alone
* Deadline for this assignment: 
 * Monday (March 12th) until midnight
 * 5 points (maximum grade = 100 points) penalization per day after deadline
* Submit your **fully executed** notebook to the grand-challenge.org platform

### Students
Please fill in this cell with your name and e-mail address. This information will be used to grade your assignment.

* <font color=#8b0000> Brian Westerweel, email address: b.westerweel@student.ru.nl</font>
* <font color=#8b0000> Jeffrey Luppes, email address: j.luppes@student.ru.nl</font>
* <font color=#8b0000> Gijs van der Meijde, email address: G.vanderMeijde@student.ru.nl</font>

We are "Team whiskey and Ritalin" on this weeks leaderboard.


# Image classification with convolutional neural networks

<img src="./figures/cifar_10.jpg" alt="CIFAR10" align="right" width="450">

In this assignment, we are going to build, train and validate **convolutional neural networks** (ConvNets).
For this purpose, we will use data from the publicly available CIFAR10 dataset.
CIFAR10 is a dataset commonly used in the community of computer vision and machine learning to benchmark new algorithms and network architectures.

CIFAR10 is a dataset that contains (small) RGB images of 32x32 px of ten different classes:
    * airplane										
    * automobile										
    * bird										
    * cat										
    * deer										
    * dog										
    * frog										
    * horse										
    * ship										
    * truck
More details can be found at this link: https://www.cs.toronto.edu/~kriz/cifar.html

We recently searched for the state-of-the-art result on this dataset, and apparently an error of **2.31** on the test set (accuracy = 97.69) has been recently reached. The approach is described in this paper: https://openreview.net/pdf?id=S1NHaMW0b

In this assignment, you will probably reach an accuracy between 60% and 70%, which is perfectly fine!

# Data
We will train our ConvNet with images from CIFAR10, a dataset of 60,000 colour images of 32x32 pixels in 10 classes. The downloaded training samples come split into 5 batches of 10,000 samples each, which is useful to do cross-validation for example. In this assignment, you will have to decide how to best split the dataset into training and validation sets. A separate test set is provided in CIFAR10, which is the same set used by other researchers to benchmark their methods.

# Tasks
We define 10 tasks in this assignment.
Although 10 tasks sound like a lot of work, you will see that (1) they are highly repetitive (meaning, you will do a lot of copy-paste across cells in the notebook), (2) you can reuse most of the things you did last week.
Like we mentioned in the lecture this week, given the experience you have gained by defining datasets and training neural networks, training convolutional neural networks is just a natural step towards using a different architectures (and some additional tricks and tools).

---

The task that we propose are the following:

### Task 1. Split data and build convolutional networks (30 points)
In this task, you will first define your training and validation set, then you will build the architecture of convolutional networks.

### Task 2. Train convolutional networks (20 points)
In this task, you will train your first model using CIFAR10, apply the trained model to the test set, and submit the results to grand-challenge. In order to define training functions and parameteres, you can reuse a lot of code developed last week.

### Task 3. Add dropout layer(s) (5 points)
In this task, you will modify the architecture of your network by adding dropout, which is implemented in Keras in the form of a layer. After that, you will repeat the training procedure and compare the results with the ones of the *plain* network.

### Task 4. Add batch normalization (5 points)
In this task, you will do something similar to task 3, but now adding batch normalization.
You will repeat the experiment and compare the performance with previous architectures.

### Task 5. Try different initialization strategies (5 points)
We have seen that at least a couple of initilization strategies are known in the literature for (convolutional) neural networks.
Several strategies are implemented in the Keras library.
Try some of them and report the results.

### Task 6. Try different nonlinearities (5 points)
The same for nonlinearities, we have seen that ```ReLU``` is some kind of default choice for ConvNets, but other strategies exists. Do experiments, report the results and compare with previous approaches.

### Task 7. Add L2 regularization (5 points)
Modify the loss function to use L2 regularization.
Again, run experiments and report results.

### Task 8. Add data augmentation (15 points + 10 bonus points)
Think of possible ways you can augment the (training) data.
You can build a new (bigger) training set, or implement some kind of data augmentation *on-the-fly*, where some patches in the mini-batch are randomly selected and augmented with a (random) operation. Think of transformations that make sense in the context of classification of natural images.

### Task 9. Try different architecture (10 points)
You can try to improve the performance by modifying the architecture, using more layers, or wider layers (same number of layers but more filters, which means more parameters). Use all the tools you have investigated so far, the optimal combination of the options you have tried in previous tasks. The goal is to get high accuracy on the validation (and therefore on the test) set!

### Task 10 (optional). Monitor the training procedure (max 10 points)
Finally, an optional task is to implement some tools to monitor the training procedure.
Examples are the analysis of statistics of activations, or visualizing the filters learned.
If done during training, visualizing filter will also nicely show how the network refines random parameters to come up with meaningful filters (especially in the first layer).

---


As done in previous assignments, in this notebook we provide some parts of code implemented.
Some other parts are not implemented, but we define the variables that will be used in functions, to help you in the development of the assignment.
Things that have been declared but not implemented are assigned a **None** value.
That is the part that you have to implement.
This means that every time you see **None**, it means that something is missing and you have to implement it.

## Reporting your results

When you are done with the assignments you will have to hand in this notebook. Make sure that for each assignment you output a summary of the network architecture and a plot of the loss/accuracy curve during training. This output will be used to grade your assignment.

## Let's get started

In [ ]:
# import libraries
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 12)
from tqdm import tnrange, tqdm_notebook
from IPython import display
import time
import tensorflow as tf
import keras
from keras import layers, models, activations
#from PIL import Image

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


## Get to know your data: Load data and define datasets

CIFAR10 contains 5 batches that can be used for training/validation, and one batch that consists of the test set.
In order to train your network, you will have to define a training set and a validation set. Do not use the test set as training data, and do not use any knowledge on the labels of the test set (being a publicly available dataset, we cannot avoid exposing the labels of the test set).


Think of the best way to split data into training and validation set. Note that the format that layers in convolutional networks like (at least in the Keras/Tensorflow libraries that we are using), is as follows:

    (n_samples, rows, cols, n_channels)

This means that each training (but also validation and test) sample needs to have four dimensions. This kind of structure (multi-dimensional array), is called a **tensor**. In practice, this format is also convenient because the first index of the tensor refers to the sample itself, so we can use:

    tensor[i]
    
to extract the i-th example.

During training, several samples will be used to update the parameters of a network. In the case of CIFAR10, if we use M samples per mini-batch, the shape of the mini-batch data is:

    (M, 32, 32, 3)

Make sure data is organized in this way, for the training, validation and test datasets.

### Download CIFAR10 data
The following cell will download the 5 batches of the CIFAR10 training dataset as well as the test set to your disk and load it in memory.

In [ ]:
from keras.utils.data_utils import get_file
origin_url = 'https://surfdrive.surf.nl/files/index.php/s/{key}/download'

# Dictionary mapping file names to download keys
files = {
    'cifar10-batch-1': '4Nee36XgbYjp3N6',
    'cifar10-batch-2': 'LVx85dLceKI5toF', 
    'cifar10-batch-3': '4FoFmeKyXa5sYr0',
    'cifar10-batch-4': 'a4o2RatC0Fa3Exb',
    'cifar10-batch-5': 'eqBJRWdkRGk025k',
    'cifar10-test': 'nEf9Z4eg7iGmWGU'
}

# Download files to local drive (only downloads the files once)
local_files = [{'name': name, 'file': get_file(name, origin=origin_url.format(key=key))} for name, key in files.items()]

# Load all batches in memory
batches = {lf['name']: pickle.load(open(lf['file'], 'rb'), encoding='latin1') for lf in local_files}

# Convert data to floats and reshape to correct format
# This is specific for CIFAR10
for batch in batches.values():
    # Images were originally used as (CxWxH), convert to (WxHxC)
    batch['data'] = (batch['data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.).astype("float32")

data_size_in=(32, 32, 3)
n_classes=10

The CIFAR dataset is now loaded in memory in the form of a Python dictionary. To get to know your data, check what are the keys of such a dictionary:

In [ ]:
print(batches.keys())

Before you define your datasets, it is useful to check the distribution of labels across batches of CIFAR10, in case some batches have skewed distributions of labels.
In order to do that, you can use visualize the histogram of labels using the function ```hist()``` of the matplotlib library:

    plt.hist()

In [ ]:
fig = plt.figure(figsize=(20, 3))
plt.subplot(1,5,1); plt.hist(batches['cifar10-batch-1']['labels']); plt.title('batch_1')
plt.subplot(1,5,2); plt.hist(batches['cifar10-batch-2']['labels']); plt.title('batch_2')
plt.subplot(1,5,3); plt.hist(batches['cifar10-batch-3']['labels']); plt.title('batch_3')
plt.subplot(1,5,4); plt.hist(batches['cifar10-batch-4']['labels']); plt.title('batch_4')
plt.subplot(1,5,5); plt.hist(batches['cifar10-batch-5']['labels']); plt.title('batch_5')

#### Question
Do you think that you have to organize batches in a specific way in order to make a training and a validation set, and would this affect the performance of your network significantly?

*The batches are pretty balanced so in this case the effect would be negligible.*

Implement now a function ```load_data``` that builds and returns the training and validation datasets. You will be using this function in your experiments later.

In [ ]:
### Replace None with your code ###
# define training, validation and test sets
def load_data():
    
    # make training set
    train_x = np.zeros((40000, 32, 32, 3)).astype("float32")
    
    ### YOUR CODE HERE ###
    val_batch = 'cifar10-batch-5'
    train_batches = ['cifar10-batch-1', 'cifar10-batch-2', 'cifar10-batch-3', 'cifar10-batch-4']
    train_data = []
    train_labels = []
    for batch in train_batches:
        for i in range(0, len(batches[batch]['data'])):
            train_data.append(batches[batch]['data'][i])
            train_labels.append(batches[batch]['labels'][i])
        
    val_data = batches[val_batch]['data']
    val_labels = batches[val_batch]['labels']
    
    for i in range(0, len(train_data)):
        train_x[i,:,:,:] = train_data[i] 
    
    # ADD DATA TO THE VARIABLE x_train
    
    train_y = train_labels
        
    # make validation set
    val_x = val_data
    val_y = val_labels

    # make test set
    test_x = batches['cifar10-test']['data']
    
    # load labels CIFAR10
    label_to_names = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

    print ("-----------------------------------------------------")
    print (label_to_names)
    print ("-----------------------------------------------------")
    print ("> shape training set tensor: {}".format(train_x.shape))
    print ("> length training labels: {}".format(len(train_y)))
    print ("-----------------------------------------------------")
    print ("> shape validation set tensor: {}".format(val_x.shape))
    print ("> length training labels: {}".format(len(val_y)))
    print ("-----------------------------------------------------")
    print ("> shape test set tensor: {}".format(test_x.shape))
    
    return train_x, train_y, val_x, val_y, test_x


# Task 1: build a simple convolutional network

In this task you create the definition of your convolutional network. You can use parts of the code of  last week as an example. If you want you can check out the **functional API** of Keras; this API enables you to make more advanced models and is more flexible to use. A guide can be found here: https://keras.io/getting-started/functional-api-guide/

Example of sequential API:
```python
from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(32, input_dim=784), activation='relu')
model.add(layers.Dense(32, activation='relu')
model.add(layers.Dense(10, activation='softmax'))

model.compile(...)
```

Example of same model using the functional API (**recommended**):

```python
from keras import models, layers

inputs = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dense(32, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(...)
```

The functional API can look a bit more complex at start in comparision to the sequential API. However, it gives you access to each individual layer and lets you more easily build up your network. The only difference between the two APIs is the way you build up your network, all other parts (training, applying a network, etc.) are the same.

For the assignments it does not matter which API you choose, **you are free to choose the sequential or functional API**. For the final assignment of the course it can be usefull to get some experience with the functional API.

### Network
Define your network builder function. 
We build a convolutional network that contains:

1. input layer
2. convolutional layer
3. max pooling layer
4. convolutional layer
5. max pooling layer
6. fully-connected layer(s)
7. soft-max layer

### Hint

1. Select the number of convolutional and max pooling layers and choose the filter size so that the input image is shrinked to 5x5 before the fully connected layers.
2. Use at least one fully connected layer between the last convolutional layer and the output layer (which is the fully connected layer with softmax nonlinearity).
3. You can reuse the code you used last week to build a fully-connected network, just remember that this time you have to use convolutional layers!

In [ ]:
# Define your network builder function. You can assume the (32, 32, 3) input size.
#
from keras.models import Model
def build_neural_network(data_size_in, n_classes):
        
    '''  This network scored 68.97% (rank 5 at the time) on Grand Challenge (300 epochs, 128 batch size
    '''
    
    inputs = layers.Input(shape=data_size_in, name='input')
    conv1 = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    max1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)
    conv2 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(max1)
    max2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(conv2)
    fulcon1 = layers.Dense(80, activation='relu')(max2)
    flatten = layers.Flatten()(fulcon1)
    softmax = layers.Dense(n_classes, activation='softmax')(flatten)

    network = Model(inputs=inputs, output=softmax)
    
    print(network.summary())
    
#     return model
    return network


## Loss function & Optimization Algorithm

Now that the architecture is defined, we have to specify the two other components of our learning framework, namely the loss function and the optimization algorithm.

Once you have defined these two components, you will have to **compile** the network that you have defined with the function ```build_neural_network```, using the function ```network.compile()```.

The ```network.compile()``` function requires the following input parameters:
1. loss function -> the loss function
2. optimizer -> the optimization algorithm
3. metrics -> the performance parameters you want to compute

You can find information about how to use the ```compile()``` function at this page: https://keras.io/getting-started/sequential-model-guide/.

### Loss
We have to define a function that, given the network, gets the predicted probability for a given input sample.
Since we are dealing with a multi-class classification problem, **categorical cross-entropy** seems a reasonable choice.

### Optimization algorithm
We also have to specify how we want to train our model. In our case, we will use "Stochastic Gradient Descent". As we have seen in the lecture this week, gradient descent algorithms need a **learning rate**, which indicates how much we step in the (opposite) direction of the gradient. We have also seen that strategy to adapt the learning rate during training are possible, but for the moment we just define a fixed learning rate. Pick a value and see what happens, you can optimize this later.

### Metrics
Since we are developing a classifier for a multi-class problem, the accuracy seems like a reasonable choice.

In the end, you need to compile your network with your settings.

In [ ]:
### Replace None with your code ###
from keras.optimizers import SGD

loss = 'categorical_crossentropy' # define the (string) loss function
learning_rate = 0.01 # pick a value for your learning rate
sgd = SGD(learning_rate) # define Stochastic Gradient Descent as the keras optimizer, which takes the learning rate as input parameter
metrics = ['accuracy'] # define (Python) list of metrics

network = build_neural_network(data_size_in, n_classes)
network.compile(loss=loss, optimizer=sgd, metrics=metrics)

### One-hot encoding

If we are using categorical cross-entropy as loss function, we need a representation of labels in the training (and later validation) data in a 'one-hot' form. This means that if we have 5 classes, the format of labels has to be the following:

* ```y_train = 1 -> [1, 0, 0, 0, 0]```
* ```y_train = 2 -> [0, 1, 0, 0, 0]```
* ```y_train = 3 -> [0, 0, 1, 0, 0]```
* ```y_train = 4 -> [0, 0, 0, 1, 0]```
* ```y_train = 5 -> [0, 0, 0, 0, 1]```

Lucky for you, Keras has implemented such a [function](https://keras.io/utils/#to_categorical). We can use that to convert the given format into a 'one-hot' format. First, check the format of labels in your dataset, then check if the function of Keras does what it is supposed to do.

In [ ]:
### Replace None with your code ###

# Load data using the function we defined above
train_x, train_y, val_x, val_y, test_x = load_data()

train_y_one_hot = keras.utils.to_categorical(train_y, num_classes=10)

val_y_one_hot = keras.utils.to_categorical(val_y, num_classes=10)

# This should print: (40000, 10)
print(train_y_one_hot.shape)
# This should print: (10000, 10)
print(val_y_one_hot.shape)

# check number of samples per class
print (np.sum(train_y_one_hot, axis=0))
print (np.sum(val_y_one_hot, axis=0))

### Check validation performance before training
All the main components required to train our network have been defined now.
However, we have seen that in order to properly monitor the behaviour of a network during training, we should check the performance (the accuracy) on a separate validation set, and see if it returns something sensible.
For this purpose, you can use the function ```network.evaluate``` in Keras, and set the ```batch_size``` to a value that fits in your GPU/CPU. 

In [ ]:
print(val_y_one_hot.shape)
scores = network.evaluate(val_x, val_y_one_hot, batch_size=128)
val_loss = scores[0]
val_acc = scores[1]
print ('Initial validation accuracy = {:.2f}%'.format(100.*val_acc))

# Task 2: Train your network
Now you can define a function that does trains the convnet by updating parameters for each mini-batch. This function will have to inculde the two main steps that we implemented last week:

  1. a pass over the training set, to update the parameters
  2. a pass over the validation set, to check the performance.
  
During training/validation, you will have to store the loss and accuracy values, in order to visualize them after each epoch in a plot that shows the learning curves. This is useful to monitor the training procedure.
Note that all these steps have been implemented in the previous assignment, you can reuse a lot of that code!

During training, we will also be saving to disk the parameters of the network wich has the best performance on the validation set. This will be stored as the file ```best_model.h5``` in the direcotry ```network_dir```, which by default is the root directory of this notebook.

We wrap the training code in a function so that you can re-use it of each of the tasks in this assignment.

In [ ]:
### Replace None with your code ###

def train_network(network, x_training, y_training, x_validation, y_validation, n_epoch, batch_size, network_filepath):

    # lists where we will be storing values during training, for visualization purposes
    tra_losses = [] # list for training loss
    tra_accs = [] # list for training accuracy
    val_losses = [] # list for validation loss
    val_accs = [] # list for validation accuracy

    # we want to save the parameters that give the best performance on the validation set
    # therefore, we store the best validation accuracy, and save the parameters to disk
    best_validation_accuracy = 0 # best validation accuracy

    for epoch in range(n_epoch):
        st = time.time()

        # Train your network
        results = network.fit(x_training, y_training, batch_size=batch_size)

        # Get training loss and accuracy
        training_loss = results.history['loss']
        training_accuracy = results.history['acc']

        # Add to list
        tra_losses.append(training_loss)
        tra_accs.append(training_accuracy)

        # Evaluate performance (loss and accuracy) on validation set
        scores = network.evaluate(x_validation, y_validation, batch_size=batch_size)     
        validation_loss = scores[0]
        validation_accuracy = scores[1]

        # Add to list
        val_losses.append(validation_loss)
        val_accs.append(validation_accuracy)

        # (Possibly) update best validation accuracy and save the network
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            network.save(network_filepath)

        # Visualization of the learning curves
        fig = plt.figure(figsize=(10, 5))
        tra_loss_plt, = plt.plot(range(len(tra_losses)), tra_losses, 'b')
        tra_accs_plt, = plt.plot(range(len(tra_accs)), tra_accs, 'c')
        val_loss_plt, = plt.plot(range(len(val_losses)), val_losses, 'g')
        val_acc_plt, = plt.plot(range(len(val_accs)), val_accs, 'r')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend([tra_loss_plt, tra_accs_plt, val_loss_plt, val_acc_plt], 
                  ['training loss', 'training accuracy', 'validation loss', 'validation accuracy'],
                  loc='center left', bbox_to_anchor=(1, 0.5))
        plt.title('Best validation accuracy = {:.2f}%'.format(100. * best_validation_accuracy))
        display.clear_output(wait=True)
        display.display(plt.gcf())
        time.sleep(.2)

## Task 2: Train your network
Now that you have defined all the parameters and the functions necessary to train and validate your network, use this cell to run your exepriment. Define a *network_name*, which will be used to (1) save the parameters of the trained network to disk and (2) save a csv file to submit to challenger. Since you will be running several experiments and reusing the same cell (copy-paste) several times, having a name for the network used in each experiment is handy!

In [ ]:
# Define batch size.
batch_size = 128

# keep this low (like 1 or 2) when just writing code, increase it to 350 or higher when
# submitting to Cartesius or running it on GPU. 

n_epoch = 2
file_dir = './'
network_filepath = os.path.join(file_dir, 'best_model_net_task_2.h5')

train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)


### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to https://ismi-cifar.grand-challenge.org/introduction/ for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_2'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 3: Add dropout layers
Modify your network and add dropout.

**Hint**: dropout is typically added to fully-connected layers, but it can be applied to convolutional layers as well.

In order to prepare and run your experiment, copy and modify previous cells to fill in the next three cells. Please do the same for the following tasks as well.

In [ ]:
def build_neural_network_drop(data_size_in, n_classes):
    
        
    '''  This network scored x on Grand Challenge (300 epochs, 128 batch size
    '''
    
    inputs = layers.Input(shape=data_size_in, name='input')
    conv1 = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    max1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)
    drop1 = layers.Dropout(0.5)(max1)
    conv2 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(drop1)
    max2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(conv2)
    drop2 = layers.Dropout(0.5)(max2)
    fulcon1 = layers.Dense(80, activation='relu')(drop2)
    flatten = layers.Flatten()(fulcon1)
    softmax = layers.Dense(n_classes, activation='softmax')(flatten)

    network = Model(inputs=inputs, output=softmax)
    
    print(network.summary())
    
    return network


In [ ]:
# Compile the network, you can copy code from the first task.

network = build_neural_network_drop(data_size_in, n_classes)
network.compile(loss=loss, optimizer=sgd, metrics=metrics)

In [ ]:
# train and validate the network
network_filepath = os.path.join(file_dir, 'best_model_net_dropout.h5')

train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)

#### Question
The learning curves changed after adding dropout. How, and why?

* By randomly 'deactivating' nodes the trainer cannot put all its trust in one node and will distribute the weight more evenly. *

### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_3'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 4: Add batch normalization
Add batch normalization to your network.

In [ ]:
def build_neural_network_batch_norm(data_size_in, n_classes):
    
    inputs = layers.Input(shape=data_size_in, name='input')
    conv1 = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    norm1 = layers.BatchNormalization(axis=1)(conv1)
    max1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(norm1)
    drop1 = layers.Dropout(0.5)(max1)
    conv2 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(drop1)
    norm2 = layers.BatchNormalization(axis=1)(conv2)
    max2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(norm2)
    drop2 = layers.Dropout(0.5)(max2)
    fulcon1 = layers.Dense(80, activation='relu')(drop2)
    flatten = layers.Flatten()(fulcon1)
    softmax = layers.Dense(n_classes, activation='softmax')(flatten)

    network = Model(inputs=inputs, output=softmax)
    
    print(network.summary())
    
    return network

In [ ]:
# Compile the network, you can copy code from the first task.

network = build_neural_network_batch_norm(data_size_in, n_classes)
network.compile(loss=loss, optimizer=sgd, metrics=metrics)

In [ ]:
# train and validate the network
network_filepath = os.path.join(file_dir, 'best_model_net_batchnorm.h5')
n_epoch = 3
train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)

### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_4'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 5: Try different initialization strategies
Try different Weight initialization strategies in your network. Make a new cell for each different strategy and retrain your network every time. Please see https://keras.io/initializers/ for examples.

In [ ]:
def build_neural_network_diff_init(data_size_in, n_classes, kernal_init, bias_init):
    
    #inputs = layers.Input(shape=data_size_in)
    inputs = layers.Input(shape=data_size_in, name='input_t5')
    conv1 = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    max1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)
    conv2 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(max1)
    max2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(conv2)
    fulcon1 = layers.Dense(80, activation='relu')(max2)
    flatten = layers.Flatten()(fulcon1)
    softmax = layers.Dense(n_classes, activation='softmax', kernel_initializer=kernal_init, bias_initializer=bias_init)(flatten)
    
    model = Model(inputs=inputs, output=softmax)
    #model.add(Dense(64, kernel_initializer='random_uniform', bias_initializer='zeros'))
    print(model.summary())
    
    return model

In [ ]:
# Compile the network, you can copy code from the first task.
networks = [];

#function to add network and init string to list as a tuple (network, init_string)
def addNetwork(network_array, kernal_init, bias_init):
    network_array.append( (build_neural_network_diff_init(data_size_in, n_classes, kernal_init, bias_init), "kernal: "+kernal_init+"   bias: "+bias_init))

addNetwork(networks, 'random_uniform', 'zeros')
addNetwork(networks, 'random_normal', 'zeros')
addNetwork(networks, 'random_uniform', 'ones')
addNetwork(networks, 'random_normal', 'ones')
addNetwork(networks, 'orthogonal', 'zeros')
addNetwork(networks, 'orthogonal', 'ones')

for network in networks:
    print("Compiling with initializations \n"+network[1])
    network[0].compile(loss=loss, optimizer=sgd, metrics=metrics)

In [ ]:
# train and validate the network
network_filepath = os.path.join(file_dir, 'best_model_net_init.h5')
for network in networks:
    print("Training with initializations \n"+network[1])
    train_network(network[0], train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)

### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_5'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 6: Try different nonlinearities
Try different nonlinearities in your network. You can view supported activations functions in the Keras documentation: https://keras.io/activations/. If you want you can also try the advanced activations; these are implemented as layers.

In [ ]:
def build_neural_network_diff_nonlin(data_size_in, n_classes):
 
    inputs = layers.Input(shape=data_size_in)
        
    inputs = layers.Input(shape=data_size_in, name='input')
    conv1 = layers.Conv2D(32, (3,3))(inputs)
    elu1 = layers.ELU(alpha=1.0)(conv1)
    
    max1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(elu1)
    conv2 = layers.Conv2D(32, (3,3), padding='same')(max1)
    elu2 = layers.ELU(alpha=1.0)(conv2)
    
    max2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(elu2)
    fulcon1 = layers.Dense(80)(max2)
    elu3 = layers.ELU(alpha=1.0)(fulcon1)
    
    flatten = layers.Flatten()(elu3)
    softmax = layers.Dense(n_classes, activation='softmax')(flatten)

    network = Model(inputs=inputs, output=softmax)
    
    print(network.summary())
    
    
    return network

In [ ]:
def build_neural_network_diff_nonlinLeaky(data_size_in, n_classes):
    # was given this week, uses Leaky Relu
    # Other options that look cool are ELU and Prelu
    inputs = layers.Input(shape=data_size_in)
    net = layers.Conv2D(filters=16, kernel_size=5)(inputs)
    net = layers.LeakyReLU(alpha=0.3)(net)
    net = layers.BatchNormalization()(net)
    net = layers.MaxPooling2D(pool_size=3)(net)
    
    net = layers.Conv2D(filters=32, kernel_size=5)(net)
    net = layers.LeakyReLU(alpha=0.3)(net)
    net = layers.BatchNormalization()(net)
    net = layers.MaxPooling2D(pool_size=3)(net)
    
    net = layers.Flatten()(net)
    net = layers.Dense(units=256)(net)
    net = layers.LeakyReLU(alpha=0.3)(net)
    net = layers.BatchNormalization()(net)
    
    net = layers.Dropout(rate=0.3)(net)
    net = layers.Dense(units=n_classes, activation='softmax')(net)
    
    model = models.Model(inputs=inputs, outputs=net)
    
    print(model.summary())
    
    return model


In [ ]:
# Compile the network, you can copy code from the first task.
network = build_neural_network_diff_nonlin(data_size_in, n_classes)
# network = build_neural_network_diff_nonlinLeaky(data_size_in, n_classes)
network.compile(loss=loss, optimizer=sgd, metrics=metrics)

In [ ]:
# train and validate the network
network_filepath = os.path.join(file_dir, 'best_model_net_nonlin.h5')

train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)

#### Question
Calculating the sigmoid function is computationally expensive. What is the other main weakness of the function?

*Your answer here.*

### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_6'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 7: L2 normalization
Add L2 regularization to your loss calculation. You can find examples at https://keras.io/regularizers/.

In [ ]:
from keras import regularizers

def build_neural_network_l2(data_size_in, n_classes):
    
    
    '''  This network scored x% (rank y at the time) on Grand Challenge (300 epochs, 128 batch size
    '''
        
    inputs = layers.Input(shape=data_size_in, name='input')
    conv1 = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01))(inputs)
    max1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)
    conv2 = layers.Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(max1)
    max2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(conv2)
    fulcon1 = layers.Dense(80, activation='relu', kernel_regularizer=regularizers.l2(0.01))(max2)
    flatten = layers.Flatten()(fulcon1)
    softmax = layers.Dense(n_classes, activation='softmax')(flatten)

    network = Model(inputs=inputs, output=softmax)
    
    print(network.summary())
    return network


In [ ]:
# Compile the network, you can copy code from the first task.

network = build_neural_network_l2(data_size_in, n_classes)
network.compile(loss=loss, optimizer=sgd, metrics=metrics)

In [ ]:
# train and validate the network
network_filepath = os.path.join(file_dir, 'best_model_net_l2.h5')

train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)

#### Question
Is it necessary to monitor the L2 loss during training? Why?

*Your answer here.*

### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_7'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 8: Add data augmentation

In this task you will have to implement different data augmentation techniques. We have created a simple framework to help you with this. You will have to implement at least three functions that augment an image using some method. Think about the task and images, and decide which augmentations could help the network to generalize.

You are free to use data augmentation libraries/frameworks (for example https://pillow.readthedocs.io or https://github.com/mdbloice/Augmentor). Feel free to change the augmentation framework below if you have any other ideas to augment the data or want to use an additional framework. Make sure to document which external software packages you use (if any). If you implement the augmentations yourself **you can get up to 10 bonus points** for this assignment.

In [ ]:
# Create (at least) three functions that take an image as input and return an augmented image
# Please give them sensible names, describing the augmentation.
def augmentation_func_1(image):
    return image

def augmentation_func_2(image):
    return image

def augmentation_func_3(image):
    return image

# Fill this array with your augmentation functions
augmentations = [augmentation_func_1, augmentation_func_1, augmentation_func_1]

# Preview your augmentations on an image
test_image = train_x[1].copy()
fig, axarr = plt.subplots(ncols=len(augmentations) + 1, nrows=1)
axarr[0].imshow((test_image))
for key, augment_func in enumerate(augmentations):
    axarr[key + 1].imshow(augment_func(test_image))
plt.show()

In [ ]:
# This is my idea (jeff). The below function uses Keras' own augmentation and is way easier...
# but has to be called differently because this thing will loop indefinitely
# so this really makes things a tad more complex for debugging with Cartisius involved.
# https://keras.io/preprocessing/image/
# see above for details
datagen = preprocessing.image.ImageDataGenerator(rotation_range=90,  #rotate images 90 degrees 
    width_shift_range=0.2,    # Shift iages to left and right
    height_shift_range=0.2,  # Shift images up and down
    horizontal_flip=True,  # flip horizontally
    vertical_flip=False)  # flip vertically
dategen.fit(x_training)


The function below takes the original data as input and returns an augmented dataset.

In [ ]:
# Function to augment data
# The ratio defines how many of the images will be augmented.
def augment_data(original_data, ratio=0.2):

    data = original_data.copy()
    
    # Apply augmentations on a random portion of the data
    for augment_func in augmentations:
        indices = np.random.choice(len(data), int(len(data) * ratio))
    
        for i in indices:
            data[i] = augment_func(data[i])
    
    return data

Our train function needs to be adapted to work with the augmentations. Think about how you can do this. When do you need to apply the augmentations?

In [ ]:
def train_network_with_augmentations(network, x_training, y_training, x_validation, y_validation, n_epoch, batch_size, network_filepath):
    
    # Copy your implementation of the previous task and extend it to support augmented images.
    
    pass

In [ ]:
network_filepath = os.path.join(file_dir, 'best_model_net_augmentations.h5')

train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)


### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_8'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

#### Question
Should the upside-down flipping be used as augmentation? Why?

*Your answer here.*

# Task 9: Try different architectures

The architectures we used so far were pretty simple. Try to build a different (deeper) architecture and see how it performs.

In [ ]:
def build_neural_network_deep_arch(data_size_in, n_classes):
    #inputs = layers.Input(shape=data_size_in)
    inputs = layers.Input(shape=data_size_in, name='input_t9')
    conv1 = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    #norm1 = layers.BatchNormalization(axis=1)(conv1)             #batch normalisation
    #max1  = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(norm1)
    drop1 = layers.Dropout(0.33)(conv1)                            #dropout
    
    conv2 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(drop1)
    #norm2 = layers.BatchNormalization(axis=1)(conv2)             #batch normalisation
    #max2  = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(norm2)
    drop2 = layers.Dropout(0.33)(conv2)                            #dropout
    
    conv3 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(drop2)
    norm3 = layers.BatchNormalization(axis=1)(conv3)             #batch normalisation
    max3  = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(norm3)
    drop3 = layers.Dropout(0.33)(max3)                            #dropout
    
    conv4 = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01))(drop3) #L2 normalisation
    #norm4 = layers.BatchNormalization(axis=1)(conv4)             #batch normalisation
    #max4  = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(norm4)
    drop4 = layers.Dropout(0.33)(conv4)                            #dropout
    
    conv5 = layers.Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(drop4) #L2 normalisation
    #max5  = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv5)
    drop5 = layers.Dropout(0.33)(conv5)                            #dropout
    
    #Add non lineairities
    conv6 = layers.Conv2D(32, (3,3))(drop5)
    #elu6 = layers.ELU(alpha=1.0)(conv6)
    #max6 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(elu6)
    drop6 = layers.Dropout(0.33)(conv6)                            #dropout
    
    conv7 = layers.Conv2D(32, (3,3), padding='same')(drop6)
    elu7 = layers.ELU(alpha=1.0)(conv7)
    #max7 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(elu7)
    drop7 = layers.Dropout(0.33)(elu7)                            #dropout
    
    fulcon8 = layers.Dense(80)(drop7)
    elu8 = layers.ELU(alpha=1.0)(fulcon8)
    
    #max3 = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3))(conv3)
    fulcon2 = layers.Dense(80, activation='relu')(elu8)
    flatten = layers.Flatten()(fulcon2)
    softmax = layers.Dense(n_classes, activation='softmax')(flatten)
    
    model = Model(inputs=inputs, output=softmax)
    print(model.summary())
    
    return model

In [ ]:
def build_neural_network_deep_arch_two(data_size_in, n_classes):
    # based on this Convnet paper: https://arxiv.org/pdf/1412.6806.pdf from 2014 (published in 2015?) by Springenberg et al
    # titled "Striving for simplicity, the all-convolutional net"
    # and the reported 84-91% result 
    # Do we get bonusses for two epicly deep networks? :D
    # I "deeply" suspect ELU would be a better choice than Relu here
    inputs = layers.Input(shape=data_size_in, name='input_t9')
    x = layers.Conv2D(96, (3,3),activation='relu', padding = 'same')(inputs)
    x = layers.Dropout(0.2)(x)                            #dropout
    x = layers.Conv2D(96, (3,3),activation='relu', padding = 'same')(x)
    x = layers.Conv2D(96, (3,3),activation='relu', padding = 'same', strides=2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = layers.Conv2D(192, (3, 3), activation='relu', padding = 'same', strides=2)(x)
    x = layers.Dropout(0.5)(x)    
    x = layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)    
    x = layers.Conv2D(192, (1, 1), activation='relu', padding='valid')(x)
    x = layers.Conv2D(10, (1, 1), activation='softmax', padding='valid')(x)
    gpl = layers.GlobalAveragePooling2D()(x)
    network = Model(inputs=inputs, output=gpl)
    
    print(network.summary())
    
    
    return network


In [1]:
# Compile the network, you can copy code from the first task.

network = build_neural_network_deep_arch(data_size_in, n_classes)
network = build_neural_network_deep_arch_two(data_size_in, n_classes)
network.compile(loss=loss, optimizer=sgd, metrics=metrics)

NameError: name 'build_neural_network_deep_arch_two' is not defined

In [ ]:
# train and validate the network
network_filepath = os.path.join(file_dir, 'best_model_net_deep.h5')

train_network(network, train_x, train_y_one_hot, val_x, val_y_one_hot, n_epoch, batch_size, network_filepath)

### Test and submit results to grand-challenge
Now you can run the network on the test set, get the predicted labels, and submit the results to grand-challenge.org for evaluation. Use the following code (copy-paste) also for future tasks.

In [ ]:
# Re-load the model found for the best accuracy; load_model function takes care of compiling again the function 
best_network=keras.models.load_model(network_filepath)

network_name = 'network_task_9'
test_prediction = best_network.predict(test_x, batch_size=128)
test_y = np.argmax(test_prediction, axis=1)

# write csv files with outputs
ho = open('./results_{}.csv'.format(network_name), 'w')
ho.write('filename, label\n')
for filename, label in zip(batches['cifar10-test']['filenames'], test_y):
    ho.write('{}, {}\n'.format(filename, label))
ho.close()

# Task 10 (optional): Tools to monitor training procedure

In [ ]:
# >>> add your code here <<<